In [1]:
print("hello world")

hello world


In [2]:
import os
import pandas as pd
import traceback


In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [135]:
from langchain.chat_models import ChatOpenAI

In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [6]:
llm = ChatOpenAI(
    model="deepseek-ai/DeepSeek-V3:fireworks-ai", 
    openai_api_base="https://router.huggingface.co/v1",
    openai_api_key=os.environ["HUGGINGFACEHUB_API_TOKEN"],
    temperature=0
)

C:\Users\Agathiyan\AppData\Local\Temp\ipykernel_24864\4077676499.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(


In [115]:
X = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    }
}


In [116]:
Y = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "review": "Review of the quality of the mcq"
}


In [117]:
prompt_mcq_generator=PromptTemplate.from_template("""Content: {Text} .You are an english expert. Given the content you have to generate {Number} MCQ's for the student with the difficulty of {Difficulty}. The response should be of the format X.\
                                                  Return the result strictly in json format (not Python dict, not Markdown, no explanations)\
                                                  format of json\
                                                  {X}""")
                                                  

In [118]:
prompt_mcq_evaluater=PromptTemplate.from_template("""Quiz: {Quiz} .You are an Mcq evaluater expert. Given the mcq your role is to review the mcq and Provide a review of less than 50 words and then correct the MCQ's if the difficulty is not on par with {Difficulty}.\
                                                  And provide the final result of the format Y. Make sure that the number of questions is {Number} and the review is given in the review section.\
                                                  Return the result strictly in JSON format (not Python dict, not Markdown, no explanations)\
                                                  format of JSON:\
                                                  {Y}""")

In [119]:
mcq_generator=LLMChain(llm=llm,prompt=prompt_mcq_generator,output_key="Quiz")
mcq_evaluater=LLMChain(llm=llm,prompt=prompt_mcq_evaluater,output_key="Review")

In [120]:
chain=SequentialChain(chains=[mcq_generator,mcq_evaluater],input_variables=["Text","Difficulty","Number","X","Y"],output_variables=["Quiz","Review"])

In [111]:
prompt_mcq_evaluater.format(Y=Y,Quiz=X,Difficulty="Hard")

KeyError: 'Number'

In [121]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../nature.pdf"
loader = PyPDFLoader(file_path)

In [122]:
docs = loader.load()
print(docs[0].page_content[:])
content=docs[0].page_content[:]

____________________________________________________________________________________________  
©2013  UGC-Infonet Digital Library Consortium                                                                 INFLIBNET Centre 
106
Nature                (http://www.nature.com) 
 
Nature retains its position as the most cited weekly science journal, with over 390,000 cites, 
an increase of almost 18,000 on last year’s count. And Nature continues to publish more 
articles than any other mu ltidisciplinary journal. For 2006 Nature’s impact factor is 26.681. 
The impact factor of a journal is calculated by dividing the number of current year citations 
to the source items published in that journal during the previous two years. It is an 
independent measure calculated by Thomson/ISI (Institute for Scientific Information), 
Philadelphia, USA. 
 
UGC-Infonet Digital Library Consortium has got subscription to only the Nature Journal, 
which can be accessed at the member Universities through their 

In [123]:
import json
result=chain.invoke({"Text":content,"Difficulty":"Hard","Number":10,"X":json.dumps(X),"Y":json.dumps(Y)})

In [127]:
print(json.loads(result["Review"][7:-3]))
print(json.loads(result["Quiz"][7:-3]))

{'1': {'mcq': 'What is the impact factor of Nature for the year 2006?', 'options': {'a': '25.681', 'b': '26.681', 'c': '27.681', 'd': '28.681'}, 'correct': 'b'}, '2': {'mcq': 'How is the impact factor of a journal calculated?', 'options': {'a': 'By dividing the number of current year citations by the total number of articles published in the journal', 'b': 'By dividing the number of current year citations to the source items published in that journal during the previous two years', 'c': 'By multiplying the number of current year citations by the total number of articles published in the journal', 'd': 'By adding the number of current year citations to the total number of articles published in the journal'}, 'correct': 'b'}, '3': {'mcq': 'Which organization calculates the impact factor of journals?', 'options': {'a': 'UGC-Infonet Digital Library Consortium', 'b': 'Nature Journal', 'c': 'Thomson/ISI (Institute for Scientific Information)', 'd': 'INFLIBNET Centre'}, 'correct': 'c'}, '4': 

# **To track the Tokens we use get_openai_callbacks**

In [129]:
from langchain.callbacks import get_openai_callback

In [130]:
with get_openai_callback() as cb:
    response=chain(
        {
            "Text": content,
            "Number": "10",
            "Difficulty": "Hard",
            "X": json.dumps(X),
            "Y": json.dumps(Y)
        }
        )

C:\Users\Agathiyan\AppData\Local\Temp\ipykernel_24864\2768777818.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response=chain(


In [131]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:3645
Prompt Tokens:1727
Completion Tokens:1918
Total Cost:0.0


In [133]:
import pandas as pd


# Remove review key if present
data=json.loads(result["Quiz"][7:-3])

# convert to df
rows = []
for qid, content in data.items():
    if qid.isdigit():  # skip 'review'
        mcq = content['mcq']
        choices = " | ".join([f"{k}: {v}" for k, v in content['options'].items()])
        correct = content['correct']
        rows.append({"mcq": mcq, "choices": choices, "correct": correct})

df = pd.DataFrame(rows)

print(df)


                                                 mcq  \
0  What is the impact factor of Nature for the ye...   
1  How is the impact factor of a journal calculated?   
2  Which organization calculates the impact facto...   
3  What is the coverage period of the Nature Jour...   
4  How many universities have access to the Natur...   
5  What is the primary method to search for artic...   
6  What is the significance of the Nature Journal...   
7  Which of the following statements about Nature...   
8  What is the role of UGC-Infonet Digital Librar...   
9  What is the primary purpose of the impact fact...   

                                             choices correct  
0      a: 25.681 | b: 26.681 | c: 27.681 | d: 28.681       b  
1  a: By dividing the number of current year cita...       b  
2  a: UGC-Infonet Digital Library Consortium | b:...       c  
3  a: 1995 onwards | b: 1996 onwards | c: 1997 on...       c  
4                      a: 70 | b: 75 | c: 80 | d: 85       c  
5  a:

In [134]:
df.to_csv("nature_mcq.csv")